In [1]:
import os
import time
import random
import re
import pandas as pd
import numpy as np

In [2]:
import jieba

In [3]:
import pickle

In [4]:
def dump_pickle(obj,fname):
    with open(fname,'wb') as file:
        pickle.dump(obj,file)
def load_pickle(fname):
    return pickle.load(open(fname,'rb'))

In [6]:
claim_df = pd.read_pickle('../data/patent_data/processed/claim_df_seg_0630.pkl')

In [27]:
patent_text = [''.join(i) for i in claim_df.seg_text.tolist()]

# 向量化 使用tfidf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorization

In [19]:
patent_tfidf_obj = load_pickle('../data/processed/patent_tfidf_obj_default_0701.pkl')

In [70]:
patent_all_tfidf_matrix = patent_tfidf_obj.transform(patent_text)

# PCA 降维对象

In [14]:
from sklearn.decomposition import TruncatedSVD

In [20]:
pca_obj = load_pickle('../data/processed/patent_pca_obj_default_0701.pkl')

In [25]:
pca_obj

TruncatedSVD(algorithm='randomized', n_components=250, n_iter=5,
       random_state=None, tol=0.0)

# 样例数据准备

In [28]:
sample_patent_data_tfidf = patent_tfidf_obj.transform(patent_text[:10000])

In [29]:
sample_patent_data_pca = pca_obj.transform(sample_patent_data_tfidf)

# 聚类 使用 k-means

In [33]:
from sklearn.cluster import k_means

In [35]:
km_tfidf = k_means(sample_patent_data_tfidf,5)

In [36]:
km_pca = k_means(sample_patent_data_pca,5)

In [37]:
km_pca

(array([[ 2.83820263e-02, -1.70306915e-02,  2.86577453e-03, ...,
          5.54199755e-03,  1.19199102e-03,  1.49053191e-04],
        [ 1.35630986e-01, -7.27477152e-02,  6.18817857e-02, ...,
         -3.87402735e-03,  8.92233791e-04,  5.09193055e-03],
        [ 1.49775592e-01, -9.20355154e-02, -1.09843952e-02, ...,
          3.30665704e-03, -1.97621281e-02,  6.29103045e-03],
        [ 9.85157907e-02, -5.78456984e-02,  3.66334644e-02, ...,
          2.90106150e-04, -2.00093270e-03,  3.47277030e-03],
        [ 6.34953713e-02, -6.42329359e-02,  2.51886400e-02, ...,
          4.60142355e-03, -4.23258529e-03,  3.14283302e-03]]),
 array([1, 0, 2, ..., 2, 1, 0], dtype=int32),
 639.3800341680208)

# 关键词抽取

使用textrank4zh包里的相关内容

In [8]:
from textrank4zh import TextRank4Keyword,TextRank4Sentence

tr4w = TextRank4Keyword()
def topkword(text,topk=20):
    tr4w.analyze(text,lower=True,window=2)
    return tr4w.keywords[:topk]

In [ ]:
claim_df['keywords'] = claim_df.text.apply(lambda x:topkword(x))

In [9]:
start_point = 96443

In [ ]:
with open('../data/patent_data/processed/claim_kword.txt','a',encoding='utf-8') as file:
    for num,_ in enumerate(claim_df.itertuples()):
        if num <start_point:
            continue
        if num % 1000==0:
            print('[{0}] Num:{1}'.format(time.ctime(),num))
        kw = topkword(claim_df.loc[num,'text'])
        kw_line = ['{0}:{1}'.format(i['word'],i['weight']) for i in kw]
        kw_line = '|'.join(kw_line)
        file.write(kw_line+'\n')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.962 seconds.
Prefix dict has been built succesfully.


In [12]:
claim_df.shape

(499763, 5)

# 聚类 使用